In [1]:
import sys
import os
sys.path.append(r"..")

from utils import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,recall_score
from gensim import models

path_build = "../../data/tencent2020/build/"
path_save = "../../data/tencent2020/save/"
pickle_path = "../../data/tencent2020/pickle"
sub_path = "../../data/tencent2020/sub"

train_preliminary_p = path_build + "train_preliminary/"

if not os.path.exists(pickle_path):
    os.mkdir(pickle_path) 
if not os.path.exists(sub_path):
    os.mkdir(sub_path)

In [3]:
user_creative_ids = pd.read_pickle(f"{pickle_path}/user_creative_ids.pickle")
user = pd.read_csv(train_preliminary_p + "user.csv", encoding='utf-8')
user_creative_ids = user_creative_ids.merge(user, how='left', on='user_id')
user_creative_ids.head()

,user_id,creative_id,age,gender
0,1,"[821396, 209778, 877468, 1683713, 122032, 7169...",4.0,1.0
1,2,"[63441, 155822, 39714, 609050, 13069, 441462, ...",10.0,1.0
2,3,"[661347, 808612, 710859, 825434, 593522, 72694...",7.0,2.0
3,4,"[39588, 589886, 574787, 1892854, 1962706, 2264...",5.0,1.0
4,5,"[296145, 350759, 24333, 43235, 852327, 1054434...",4.0,1.0


In [4]:
user_creative_ids['creative_id'] = user_creative_ids['creative_id'].map(lambda x: [str(i) for i in x])

In [5]:
user_creative_ids['creative_id'].head()

0    [821396, 209778, 877468, 1683713, 122032, 7169...
1    [63441, 155822, 39714, 609050, 13069, 441462, ...
2    [661347, 808612, 710859, 825434, 593522, 72694...
3    [39588, 589886, 574787, 1892854, 1962706, 2264...
4    [296145, 350759, 24333, 43235, 852327, 1054434...
Name: creative_id, dtype: object

Word2Vec参数：  
hs ({0, 1}, optional) – 1: 采用hierarchical softmax训练模型; 0: 使用负采样。  
1.skip-gram （训练速度慢，对罕见字有效），CBOW（训练速度快）。一般选择Skip-gram模型；  

2.训练方法：Hierarchical Softmax（对罕见字有利），Negative Sampling（对常见字和低维向量有利）；  

3.欠采样频繁词可以提高结果的准确性和速度（1e-3~1e-5）  

4.Window大小：Skip-gram通常选择10左右，CBOW通常选择5左右。  

In [ ]:
# Gen W2V Vector
w2v = models.Word2Vec(user_creative_ids['creative_id'].values, sg=1, size=200, window=10, workers=20,hs=1, min_count=25)
w2v.wv.save_word2vec_format(f"{path_save}/creative_id_w2v200.model")

#w2v = models.KeyedVectors.load_word2vec_format(f"{path_save}/creative_id_w2v200.model")

In [ ]:
def embedding_mean(ll):
    id_list = []
    for l in ll:
        if l in w2v:
            id_list.append(w2v[l])
    id_array = np.array(id_list).mean(axis=0)
    return id_array
user_creative_ids_matrix =  np.matrix(np.array([user_creative_ids['creative_id'].map(lambda x : embedding_mean(x))])[0])

In [ ]:
dd = pd.DataFrame(user_creative_ids_matrix)
dd.head()

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'n_estimators': 1000,
    'metric': 'mae',
    'learning_rate': 0.01,
    'min_child_samples': 5,
    'min_child_weight': 0.01,
    'subsample_freq': 1,
    'num_leaves': 31,
    'max_depth': -1,
    'subsample': 0.8,
    'colsample_bytree': 0.6,
    'reg_alpha': 0,
    'reg_lambda': 5,
    'verbose': -1,
    'random_state': 4590,
    'n_jobs': 4,
}

In [ ]:
cv = True
model_type = 'lgb'
seed = 2020
train_filter = 700000
train_x = dd.values[0:train_filter]
train_y = user_creative_ids['age'][0:train_filter]
test_x =  dd.values[train_filter:900000]
if cv:
    n_fold = 5
    count_fold = 0
    preds_list = list()
    oof = np.zeros(train_x.shape[0])
    kfolder = KFold(n_splits=n_fold, shuffle=True, random_state=seed)
    kfold = kfolder.split(train_x, train_y)
    for train_index, vali_index in kfold:
        print("training......fold",count_fold)
        count_fold = count_fold + 1
        k_x_train = train_x[train_index]
        k_y_train = train_y[train_index]
        k_x_vali = train_x[vali_index]
        k_y_vali = train_y[vali_index]
        if model_type == 'lgb':
            lgb_model = lgb.LGBMRegressor(**lgb_params)
            lgb_model = lgb_model.fit(k_x_train, k_y_train, eval_set=[(k_x_vali, k_y_vali)],
                                      early_stopping_rounds=200, verbose=False, eval_metric="mae",
                                      )
            k_pred = lgb_model.predict(k_x_vali, num_iteration=lgb_model.best_iteration_)
            pred = lgb_model.predict(test_x, num_iteration=lgb_model.best_iteration_)
        preds_list.append(pred)
        oof[vali_index] = k_pred

In [ ]:
preds_columns = ['preds_{id}'.format(id=i) for i in range(n_fold)]
preds_df = pd.DataFrame(data=preds_list)
preds_df = preds_df.T
preds_df.columns = preds_columns
preds = list(preds_df.mean(axis=1))

In [ ]:
preds22 = np.rint(preds)
preds22 = np.clip(preds22, 1, 10)
gender_accuracy = accuracy_score(user_creative_ids['age'][train_filter:900000], preds22)
gender_accuracy,preds22.shape

(0.904855, (200000,))  
(0.311965, (200000,))



In [ ]:
preds22.min(),preds22.max()

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg_model = reg.fit(dd.values[0:train_filter], user_creative_ids['age'][0:train_filter])
reg_pred = reg_model.predict(test_x)


In [ ]:
preds22 = np.rint(reg_pred)
preds22 = np.clip(preds22, 1, 10)
gender_accuracy = accuracy_score(user_creative_ids['age'][train_filter:900000], preds22)
gender_accuracy,preds22.shape

(0.907675, (200000,))  
 (0.29759, (200000,))
